# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.autism.org/symptoms-of-autism/'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN
...,...,...,...,...,...
110,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN
111,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN
112,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN
113,What can I do about my children’s stress?,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [6]:
x = soup.find('div',{'class':'fusion-text fusion-text-2 nitro-offscreen'}).text

In [7]:
l1=[]
for i in soup.find_all('h2'):
    l1.append(i.text.strip())
l1

['What are the Symptoms of Autism?',
 'Conditions that can occur with autism',
 'When do symptoms of autism appear?']

In [8]:
l2=[]
for i in range(len(l1)):
    print(l1[i])
    print('-'*50)
    start = x.index(l1[i])

    if(i!=len(l1)-1):
        end = x.index(l1[i+1])
    if(i==len(l1)-1):
        end = len(x)
       
    ans = x[start:end]
    ans = ans.split(l1[i])[1]
    ans = ans.strip()
    print(ans)
    print('*'*100)
    
    l2.append(ans)

What are the Symptoms of Autism?
--------------------------------------------------
While autism can only be diagnosed by a healthcare professional, this list of common symptoms associated with autism can help parents and caregivers know what to watch for. Communication and Social Symptoms   Pragmatic Language – Most individuals on the autism spectrum have difficulty communicating with other people. This often becomes apparent during early childhood. Any delays in speech development and nonverbal communication should be evaluated by a qualified specialist.   Eye Contact and Nonverbal Communication – Poor and avoidance of eye contact are common symptoms among those with autism. Other non-verbal communication difficulties may include recognizing and using facial expressions, physical gestures, and overall body language.   Tone of Voice – Some people with autism may have difficulty regulating or modulating their tone of voice. As a result, they may speak too loudly, too quietly, and/or wi

In [9]:
for i in soup.find_all('h3'):
    k = i.text.strip()
    for j in range(len(l2)):
        if(k in l2[j]):
            l2[j] = l2[j].replace(k,'\n\n'+k)

In [10]:
for i in range(len(l2)):
    print(i,l1[i])
    print('-'*50)
    print(l2[i])
    print('*'*100)

0 What are the Symptoms of Autism?
--------------------------------------------------
While autism can only be diagnosed by a healthcare professional, this list of common symptoms associated with autism can help parents and caregivers know what to watch for. 

Communication and Social Symptoms   Pragmatic Language – Most individuals on the autism spectrum have difficulty communicating with other people. This often becomes apparent during early childhood. Any delays in speech development and nonverbal communication should be evaluated by a qualified specialist.   Eye Contact and Nonverbal Communication – Poor and avoidance of eye contact are common symptoms among those with autism. Other non-verbal communication difficulties may include recognizing and using facial expressions, physical gestures, and overall body language.   Tone of Voice – Some people with autism may have difficulty regulating or modulating their tone of voice. As a result, they may speak too loudly, too quietly, and/o

# Check which questions are similar

In [15]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [16]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [17]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [18]:
sim(l1)

Website -->  What are the Symptoms of Autism? (Index 0)
100 questions list -->  Are there other diseases that have the same symptoms of autism? (Index 31)
0.6666666666666666
****************************************************************************************************
Website -->  Conditions that can occur with autism (Index 1)
100 questions list -->  What is Autism? (Index 4)
0.3333333333333333
****************************************************************************************************
Website -->  When do symptoms of autism appear? (Index 2)
100 questions list -->  Are there other diseases that have the same symptoms of autism? (Index 31)
0.5
****************************************************************************************************


In [ ]:
# Website, 100Q
# 0, 36


In [27]:
l2[2]

'Autism symptoms in children may be recognized as early as 18 months. More recently, adults are being diagnosed with autism, and this is likely due to increased awareness and screening during medical appointments. If you suspect that your son or daughter may have autism, it’s time to talk to your doctor. Interventions can make a big difference for both your child’s health and development. Learn about autism treatments for children and adults.'

In [28]:
for i,j in enumerate(df['Question']):
    if('when' in j):
        print(i,j)

21 What medical tests should the doctor perform when making a diagnosis of Autism?
90 What is a "Letter of Intent" and why is it necessary when creating a special needs trust?
102 I spent time and money childproofing my home when my son was an infant? Do I have to change anything now that he has been diagnosed as autistic?


In [29]:
# Directly add answer to existing dataframe
l4 = [[0, 36]]

for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

C:\Users\aparg\anaconda3\envs\ASD_scraping\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [30]:
l1

['What are the Symptoms of Autism?',
 'Conditions that can occur with autism',
 'When do symptoms of autism appear?']

In [31]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

['Conditions that can occur with autism', 'When do symptoms of autism appear?']

In [32]:
# select answers to questions in list 'l'
l2 = [l2[i] for i in f]

In [33]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.autism.org/symptoms-of-autism/
0,Conditions that can occur with autism,Several medical conditions are associated with...
1,When do symptoms of autism appear?,Autism symptoms in children may be recognized ...


In [34]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3.head(10)

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN
5,What are Screening Tools for Autism?,NaN,Early intervention can make a big difference f...,NaN,NaN
6,How Common Is It?,NaN,"For many years a diagnosis of autism was rare,...",NaN,NaN
7,What are the Signs of Autism?,NaN,Many signs can indicate that an individual may...,NaN,NaN
8,What should I do next?,NaN,When preparing to talk to your medical provide...,"Attend one or more parent support groups, o...",NaN
9,How can you tell Autism from Asperger’s Syndrome?,NaN,NaN,Asperger syndrome was phased out with the int...,NaN


In [36]:
df3.iloc[:,[0,-1]].dropna()

,Question,https://www.autism.org/symptoms-of-autism/
27,When do symptoms of autism appear?,Autism symptoms in children may be recognized ...
28,Conditions that can occur with autism,Several medical conditions are associated with...
29,What are some of the symptoms of Autism a Pare...,While autism can only be diagnosed by a health...


In [37]:
df3.shape

(117, 5)

# Save Output

In [38]:
df3.to_excel('Excel/Source2_3.xlsx',index=False)